# Kernel Abstractions

El paquete `KernelAbstractions.jl` te permite escribir kernels similares a un kernel de GPU que se puede ejecutar en distintos dispositivos. Su intención es ser una biblioteca mínima con buen rendimiento que permita escribir código sencillo y heterogéneo. Es una alternativa a utilizar el paquete `CUDA.jl` que además permite ejecutar los mismos kernels en CPU y en GPU.

El uso de kernels para GPU proviene del hecho de que hoy en día, el uso de tarjetas gráficas para cómputo científico ha demostrado ser bastante eficiente.

## ¿Cómo programar un GPU?
En general se hace debe tener esto en cuenta:
* Modelo SPMD (_Single Process Multiple Data_)
* Se programa con una perspectiva de **camino** (llamado hilo por Nvidia)
* Un grupo de caminos se ejecutan de manera conjunta (warp/vector)
* La ejecución se organiza en bloques/grupos que pueden acceder a recursos compartidos
* La GPU se basan en el rendimiento
* Se debe tener en cuenta la arquitectura de la memoria y el uso de ancho de banda al diseñar un kernel


In [1]:
using KernelAbstractions, CUDAKernels
using CUDA
using Test

Para escribir un kernel usando `KernelAbstractions.jl` es necesario declarar una función usando el macro `@kernel`

In [12]:
@kernel function mul2(A)
    I = @index(Global)
    A[I] = 2 * A[I]
end

mul2 (generic function with 5 methods)

Para lanzar el kernel es necesario invocar la función con el **primer parámetro** siendo el **dispositivo donde ejecutar el kernel**. El **segundo argumento** es el **tamaño del grupo de trabajo**, y el **tercero** es un rango `ndrange` estático. El segundo y tercer argumento son opcionales.

Después de instanciar el kernel lo puedes lanzar llamando el kernel con los argumentos correctos y palabras clave para configurar el lanzamiento del kernel. El lanzamiento de los kernels es asíncrono, por lo que para ver los resultados es necesario _capturar el evento_ y _esperar a que termine_.

El `ndrange` se parte en bloques, cada bloque es ejecutado por **un grupo de trabajo**. Por ejemplo, funciona bien para:
* Grupo de trabajo = (4, 4)
* ndrange = (16, 16)

In [4]:
A = ones(32,32)
kernel = mul2(CPU(),16)
event = kernel(A,ndrange=size(A))
wait(event)
@show A

32×32 Matrix{Float64}:
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 ⋮                        ⋮              ⋱  ⋮                        ⋮    
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     

Para lanzar el mismo kernel con la GPU es necesario indicar `CUDADevice()` como dispositivo, así como utilizar el tipo de datos adecuado como argumento.

In [15]:
B = CUDA.ones(1024,1024)
kernel2 = mul2(CUDADevice(),16)
event = kernel2(B,ndrange=size(B))
wait(event)
@show B


Excessive output truncated after 4195339 bytes.

B = 

1024×1024 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 ⋮                        ⋮              ⋱                 ⋮              
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.

## Lenguaje de kernel

En una función que contenga la macro `@kernel` se puede utilizar el lenguaje de kernel incluíso en el paquete:

* `@Const`: Permite declarar un argumento que es constante. Es decir, no se escribe como parte del kernel ni hace referencia a otra memoria en el kernel.

* `@index`: Permite obtener el índice del entorno dentro de un kernel. Puedes obtener tanto un índice lineal como uno cartesiano.
    ### Granularidad del Índice
    * `Global`: Usado para acceder a memoria global.
    * `Group`: El índice del grupo de trabajo.
    * `Local`: El índice dentro del grupo de trabajo.
    ### Tipo de Índice
    * `Linear`: Produce un `Int64` que se usa para acceder de manera lineal a la memoria.
    * `Cartesian`: Produce un `CartesianIndex{N}` para acceder a la memoria.
    * `NTuple`: Produce un `NTuple{N}` para acceder a la memoria.
El argumento or defecto es `Linear`.

```Julia
    @index(Global, Linear)
    @index(Global, Cartesian)
    @index(Local, Cartesian)
    @index(Group, Linear)
    @index(Local, NTuple)
    @index(Global)
```

* `@localmem`: Declara una memoria local para un grupo de trabajo.

* `@private`: Declara memoria que es local para cada elemento en el grupo de trabajo. Se puede usar de manera segura para sincronizar los hilos.

* `@synchronize`: Después de esta sentencia todas las lecturas y escrituras a memoria local y global de cada hilo en el grupo de trabajo.

* `@print`: Sirve para imprimir dentro del kernel.

## Ejemplos:


In [3]:
@kernel function copy_kernel!(A, @Const B)
    I = @index(Global)
    @inbounds A[I] = B[I]
end

copy_kernel! (generic function with 5 methods)

In [4]:
function mycopy!(A::Array, B::Array)
    @assert size(A) == size(B)
    kernel = copy_kernel!(CPU(), 8)
    kernel(A,B, ndrange=length(A))
end

mycopy! (generic function with 1 method)

In [7]:
A = zeros(128,128)
B = ones(128,128)
event = mycopy!(A,B)
wait(event)
@test A == B

Test Passed
  Expression: A == B
   Evaluated: [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0] == [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]

In [10]:
if has_cuda() && has_cuda_gpu()
    function mycopy!(A::CuArray, B::CuArray)
        @assert size(A) == size(B)
        copy_kernel!(CUDADevice(), 256)(A, B, ndrange=length(A))
    end

    A = CuArray{Float32}(undef, 1024)
    B = CUDA.ones(Float32, 1024)
    event = mycopy!(A,B)
    wait(event)
    @test A == B
end

Test Passed
  Expression: A == B
   Evaluated: Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] == Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

## Comparación entre CUDA.jl y KernelAbstractions.jl

En general, uno puede usar ambas librerías para construir kernels para GPU; sin embargo, los kernels construidos con `CUDA.jl` no pueden ejecutarse con el CPU, mientras que con `KernelAbstractions.jl` sí es posible. Los tipos de datos que usan ambos son los mismos (`CuArray`), a excepción de la manera de declarar memoria compartida. Para mostrar la diferencia de uso de ambos paquetes, crearemos el mismo programa usando ambas bibliotecas. El programa tiene como objetivo obtener la matriz transpuesta de una matriz, primero realizado de manera "ingenua", y luego usando memoria compartida para mejorar el uso de memoria y ancho de banda del GPU.

### Matriz Transpuesta Ingenua

In [2]:
using KernelAbstractions, CUDA, CUDAKernels

@kernel function transpose_kernel_nv!(A, @Const B)
    I,J = @index(Global, NTuple)
    @inbounds A[I,J] = B[J,I]
end

transpose_kernel_nv! (generic function with 5 methods)

In [6]:
B  = CUDA.rand(64,64)
A  = similar(B)
Ax = similar(B)

const transpose! = transpose_kernel_nv!(CUDADevice(), (32,32))
event = transpose!(A,B, ndrange=size(A))
wait(event)
A

64×64 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.741041    0.309143   0.423951   …  0.499337   0.669243   0.72744
 0.562824    0.628273   0.909818      0.431117   0.965646   0.50594
 0.746724    0.646066   0.99963       0.457733   0.984296   0.368442
 0.981253    0.68407    0.268391      0.178862   0.0505773  0.0209151
 0.024574    0.173985   0.847356      0.900485   0.377433   0.327913
 0.775626    0.930416   0.482024   …  0.111625   0.724642   0.966281
 0.955156    0.779585   0.523372      0.998777   0.26394    0.160348
 0.825587    0.189353   0.0402689     0.705045   0.576185   0.148098
 0.894214    0.496281   0.22742       0.310066   0.82547    0.315577
 0.00576556  0.255806   0.202438      0.982087   0.862935   0.322455
 0.00299558  0.46784    0.326191   …  0.800332   0.52368    0.0849368
 0.749366    0.453862   0.0972514     0.461876   0.974266   0.968448
 0.277551    0.89669    0.226138      0.945546   0.777721   0.132211
 ⋮                                 ⋱                 

In [5]:
B 

64×64 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.76063    0.982444   0.238076   0.486369   …  0.124105  0.257197  0.021197
 0.724946   0.0493957  0.904647   0.79699       0.450665  0.879822  0.31932
 0.379871   0.584883   0.533684   0.179053      0.407346  0.41819   0.158003
 0.0361741  0.208313   0.488369   0.0559797     0.357504  0.620683  0.617088
 0.120144   0.29577    0.684046   0.521824      0.450657  0.019757  0.747217
 0.633078   0.406814   0.0352185  0.446812   …  0.950219  0.410186  0.272251
 0.404653   0.473751   0.0512604  0.102672      0.839958  0.566055  0.96515
 0.839305   0.951306   0.407288   0.66823       0.630668  0.529566  0.275367
 0.15874    0.230324   0.895295   0.558798      0.815378  0.28682   0.915493
 0.0393457  0.225271   0.672282   0.519074      0.645698  0.668697  0.185113
 0.302265   0.883036   0.0582322  0.545559   …  0.243013  0.949003  0.416128
 0.118286   0.813103   0.7596     0.115883      0.24661   0.762576  0.642048
 0.606179   0.92375    0.237

In [7]:
function matriz_transpose_cd_nv!(A, B)
    indexX = threadIdx().x + (blockIdx().x - 1) * blockDim().x 
    indexY = threadIdx().y + (blockIdx().y - 1) * blockDim().y
    
    for j in 0:blockDim().y:blockDim().x-1
        A[indexX,indexY+j] = B[indexY + j, indexX]
    end

    return nothing
end


matriz_transpose_cd_nv! (generic function with 1 method)

In [8]:
dimGrid  = (Int(64/32),Int(64/32),1)
dimBlock = (32,8,1)

@cuda threads=dimBlock  blocks=dimGrid matriz_transpose_cd_nv!(Ax,B)

CUDA.HostKernel{typeof(matriz_transpose_cd_nv!), Tuple{CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}}}(matriz_transpose_cd_nv!, CuFunction(Ptr{Nothing} @0x0000000008dd40e0, CuModule(Ptr{Nothing} @0x0000000008c91b20, CuContext(0x000000000483c3f0, instance dbbb8d4a098e98e2))), CUDA.KernelState(Ptr{Nothing} @0x00007faa32400000))

In [9]:
Ax

64×64 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.741041    0.309143   0.423951   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.562824    0.628273   0.909818      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.746724    0.646066   0.99963       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.981253    0.68407    0.268391      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.024574    0.173985   0.847356      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.775626    0.930416   0.482024   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.955156    0.779585   0.523372      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.825587    0.189353   0.0402689     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.894214    0.496281   0.22742       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00576556  0.255806   0.202438      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00299558  0.46784    0.326191   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.749366    0.453862   0.0972514     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.277551    0.89669    0.226138      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮            

In [10]:
A == Ax

false

## Matriz transpuesta haciendo uso de memoria compartida

### Kernel Abstractions

In [41]:
import KernelAbstractions.Extras: @unroll
@kernel function transpose_coalesced(A, @Const(B))
    #TILE_DIM   = @uniform groupsize()[1]
    #BLOCK_ROWS = @uniform groupsize()[2]
    TILE_DIM   = 32
    BLOCK_ROWS = 8
    tile = @localmem eltype(A) (TILE_DIM+1,TILE_DIM)

    i, j  = @index(Local,NTuple)
    gi,gj = @index(Group,NTuple)

    I = (gi-1) * TILE_DIM + i #No podemos usar @index(Global)
    J = (gj-1) * TILE_DIM + j #Porque el rango nd está comprimido

    @unroll for k in 0:BLOCK_ROWS:(TILE_DIM-1)
        @inbounds tile[i,j+k] = B[I,J+k]
    end

    @synchronize

    I = (gj-1) * TILE_DIM + i
    J = (gi-1) * TILE_DIM + j

    @unroll for k in 0:BLOCK_ROWS:(TILE_DIM-1)
        @inbounds A[I, J+k] = tile[j+k,i]
    end
end

In [39]:
function ka_transpose(
    A::CuMatrix, 
    B::CuMatrix, 
    ::Val{TILE_DIM}=Val(32),
    ::Val{BLOCK_ROWS}=Val(8);
    dependencies=nothing
    ) where {TILE_DIM, BLOCK_ROWS}

    @assert TILE_DIM % BLOCK_ROWS == 0
    @assert size(A) == size(B) && size(A,1) == size(A,2)
    @assert size(A,1) % TILE_DIM == 0

    block_factor = div(TILE_DIM,BLOCK_ROWS)
    ndrange = (size(A,1), div(size(A,2), block_factor))

    kernel = transpose_coalesced(CUDADevice(), (TILE_DIM, BLOCK_ROWS))
    event  = kernel(A,B, ndrange=ndrange, dependencies=dependencies)
    return event
end

ka_transpose (generic function with 3 methods)

In [42]:
B = CUDA.rand(128,128)
A = similar(B)
event = ka_transpose(A,B)
wait(event)
B

128×128 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.279239   0.0887743  0.269508    …  0.539111   0.0753977  0.0436286
 0.425819   0.492812   0.00604024     0.78792    0.795262   0.903381
 0.942517   0.320622   0.890359       0.429415   0.306786   0.24197
 0.184102   0.737187   0.505498       0.800187   0.787036   0.640905
 0.326671   0.367217   0.355659       0.0473809  0.0667441  0.21481
 0.583603   0.556011   0.183512    …  0.465528   0.911416   0.977237
 0.92897    0.868435   0.139299       0.987325   0.206076   0.974948
 0.714868   0.892955   0.0330051      0.0626288  0.221134   0.403528
 0.0928244  0.388599   0.145105       0.429505   0.722478   0.81937
 0.774083   0.218699   0.0855474      0.0053203  0.776218   0.0392996
 0.460299   0.285975   0.21818     …  0.0450912  0.687131   0.608314
 0.375465   0.5792     0.0607396      0.43391    0.0103343  0.27804
 0.782234   0.551244   0.61692        0.217464   0.275764   0.550559
 ⋮                                 ⋱  ⋮              

In [43]:
A

128×128 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.279239    0.425819    0.942517  …  0.994761   0.660561   0.34395
 0.0887743   0.492812    0.320622     0.285986   0.182186   0.459064
 0.269508    0.00604024  0.890359     0.362094   0.0503142  0.568918
 0.0673942   0.00120558  0.39774      0.338843   0.324891   0.710175
 0.843009    0.542222    0.1444       0.714063   0.115417   0.626225
 0.441298    0.829613    0.6867    …  0.47522    0.864212   0.83372
 0.554936    0.631655    0.776222     0.914168   0.457769   0.378902
 0.793884    0.39826     0.328077     0.47758    0.11679    0.335602
 0.418474    0.534767    0.417985     0.324148   0.942557   0.927066
 0.299047    0.421913    0.747633     0.0869144  0.437664   0.539074
 0.831477    0.450095    0.253783  …  0.21658    0.957726   0.638393
 0.74802     0.875936    0.855633     0.732257   0.855873   0.700877
 0.0536358   0.011064    0.515682     0.071261   0.842699   0.603946
 ⋮                                 ⋱  ⋮              

### CUDA.jl

In [70]:
function matrix_transpose_shared!(input, output)
    sharedMemory = CUDA.CuStaticSharedArray(Float64,(32 + 1,32))

    #global index
    indexX = threadIdx().x + (blockIdx().x-1) * blockDim().x
    indexY = threadIdx().y + (blockIdx().y-1) * blockDim().y 

    #transposed global index
    tindexX = threadIdx().x + (blockIdx().y-1) * blockDim().x 
    tindexY = threadIdx().y + (blockIdx().x-1) * blockDim().y 

    #local index 
    localIndexX = threadIdx().x 
    localIndexY = threadIdx().y 
    index = (indexY-1) * 64 + indexX 
    transposedIndex = (tindexY-1) * 64 + tindexX

    #transposed the matrix in shared memory 
    #global memory is read in coalesced fashion

    sharedMemory[localIndexX,localIndexY] = input[index]
    
    CUDA.sync_threads()
    output[transposedIndex] = sharedMemory[localIndexY,localIndexX]
    return 
end

matrix_transpose_shared! (generic function with 1 method)

In [71]:
a = CUDA.rand(64,64)
b = similar(a)
@cuda threads=(32,32,1) blocks=(div(64,32),div(64,32),1) matrix_transpose_shared!(a,b)


CUDA.HostKernel{typeof(matrix_transpose_shared!), Tuple{CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}}}(matrix_transpose_shared!, CuFunction(Ptr{Nothing} @0x000000000b9252c0, CuModule(Ptr{Nothing} @0x000000000ba20390, CuContext(0x000000000483c3f0, instance dbbb8d4a098e98e2))), CUDA.KernelState(Ptr{Nothing} @0x00007faa32400000))

In [72]:
Array(a)' == Array(b)

true